<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><h1 style="text-align:center;padding-top:3px">Configuration Info</h1></div>    

Standard PARADIM SciServer container needs two libraries installed:

```bash
conda install paramiko
pip install hublib
```

You can do that by opening a terminal within your container (New menu on the right side of the home browser window) 

Login parameters are stored in the file paradim_parameters.py.  The username and password will have to be edited in that file.

In [ ]:
from IPython.display import display
import math
import time
import re
import numpy as np
from paradim_parameters import PARADIM
from paradim_parameters import PARADIM_UI
from paradim_utils import GetPbs, getStatus, isfloat, fileExist
from paradim_connect import ShellHandler
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.display import clear_output
import plotly.graph_objs as go
init_notebook_mode(connected=True)


<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><h1 style="text-align:center;padding-top:3px">Maryland Advanced Research Computing Center Connection</h1></div>    


This tutorial we will perform calculations at the Maryland Advanced Research Computing Center
(MARCC), In order to connect to MARCC we need to use the `two-factor authentication' protocol.

For this you will need:
- The username sent to you by MARCC via email
- The password sent to you by MARCC via email
- The Google Authenticator Token

Please fill out all the fields below and press the 'Connect' Button, if a connection is succesfull stablished the message "Connection created successfully" would be displayed.

If you get the message "ERROR: A new connection is required" on any of the steps below, you must update the information with a new google authenticator token, and press the button connect again.

Each time this notebook is loaded, this would assign a new working folder to store results. If you know the folder name containing previous results and skip some of the steps, please change it.


In [ ]:
def CreateConnection( event ):    
    global PARADIM_UI, PARADIM
    try :
        PARADIM_UI['s0']['status'].value = "Connecting..."            
        PARADIM['SESSION'] = ShellHandler( PARADIM_UI['s0']['code'].value, 
                                           PARADIM_UI['s0']['user'].value, 
                                           PARADIM_UI['s0']['pwd'].value, 
                                           PARADIM['SERVER'], 
                                           PARADIM['PORT'])
        folder_name = PARADIM_UI['s0']['folder'].value
        if PARADIM['SESSION'] is not None and PARADIM['SESSION'].is_active():                
            stdin, stdout, stderr, command = PARADIM['SESSION'].execute("mkdir $HOME/" + folder_name);
            stdin, stdout, stderr, command = PARADIM['SESSION'].execute("cd $HOME/" + folder_name);            
            PARADIM_UI['s0']['status'].value = "Connection created successfully"
        else:
            PARADIM_UI['s0']['status'].value = "ERROR: There has been an issue with the connection"            
    except:
        PARADIM_UI['s0']['status'].value = "ERROR: There has been an issue with the authentication"
        
PARADIM_UI['s0']['button'].on_click(CreateConnection)
display(PARADIM_UI['s0']['display'])

<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><h1 style="text-align:center;padding-top:3px">Introduction and Goal</h1></div>

The rare earth lanthanide nickelates (ReNi$O_{3}$) with distorted-perovskite structure are a model system of strongly correlated electrons with a charge-transfer gap. The discovery of switching behavior is promising for electrochemical applications. High-quality epitaxial thin films and multilayers have enabled the design of a polar metal and the discovery of antiferromagnetic order, due to correlations between dimensionality and orbital order.

In this tutorial we will use Density Functional Theory (DFT) calculations to estimate the stability of Ruddlesden-Popper phases of nickelates:   $LaNiO_{3}$, $La_{2}NiO_{4}$, $La_{3}Ni_{2}O_{7}$ and $La_{4}Ni_{3}O_{10}$ using DFT calculation. 

First, we need to calculate the total energy of each of these structure $La_{4}Ni_{3}O_{10}$, $La_{3}Ni_{2}O_{7}$, $La_{2}NiO_{4}$, $LaNiO_{3}$, $La_{2}O_{3}$, $NiO$ and $O_{2}$, using DFT calculation implemented in Quantum Espresso software package https://www.paradim.org/toolbox/theory/quantum_espresso/tutorials.
<img src="Image1.png">

For a standard energy calculation using DFT, we need to test the convergence of certain parameters for more details please follow the link: https://www.paradim.org/sites/default/files/2019-05/Convergence_Parameters_of_DFT_Calculations.pdf. For the purpose of this tutorial, to keep the calculation short, we use a modest k-point grid of 4 × 4 × 4, ecutwfc = 30.0 Ry, and conv_thr = 1.0 × 10-8.


In [ ]:
def CalculateDFTEnergy ( event ):
    global PARADIM_UI, PARADIM
    
    files = [ "O.pz-n-rrkjus_psl.0.1.UPF", "Ni.pz-n-rrkjus_psl.0.1.UPF", "La.pz-spfn-rrkjus_psl.1.0.0.UPF" ]
        
    if PARADIM['SESSION'] is not None and PARADIM['SESSION'].is_active():        
        id_name = PARADIM['TUTORIAL_NAME'] + '_introduction'
        pbs_file = id_name + ".pbs"
        folder_name = PARADIM_UI['s0']['folder'].value
        PARADIM_UI['s1']['status'].value = 'Q'
        stdin, stdout, stderr, command = PARADIM['SESSION'].execute("cd $HOME/" + folder_name);            
        for upf in files:
            if (fileExist(PARADIM['SESSION'], upf) is False):
                PARADIM['SESSION'].execute("wget http://www.quantum-espresso.org/upf_files/" + upf, PARADIM_UI['s1']);
        for k, v in PARADIM_UI['s1']['base_mat'].items():
            inputdeck = PARADIM_UI['s1']['input_' + k].value.replace('\n', '\\n')
            PARADIM['SESSION'].execute("printf \"" + inputdeck + "\" > " + k + ".in", PARADIM_UI['s1']);
            if (fileExist(PARADIM['SESSION'], k+'.out') is False):
                PARADIM_UI['s1']['status'].value =  "[" + k + "] Q"
                pbs_file = k + ".pbs"
                pbs = GetPbs(PARADIM['PBS_TEMPLATE'], k, 'mpiexec -np 12 pw.x -npool 1 < ' + k + '.in > ' + k + '.out\n').replace('\n', '\\n')
                PARADIM['SESSION'].execute("printf \"" + pbs + "\" > " + pbs_file, PARADIM_UI['s1']);
                stdin, stdout, stderr, command = PARADIM['SESSION'].execute("qsub " + pbs_file, PARADIM_UI['s1']);
                try:
                    code = stdout[len(stdout)-1].strip('\n')
                    if (code.isdigit()):
                        PARADIM_UI['s1']['job_id'].value = code
                        status = 'Q'   
                        while status != 'E' and status != 'X' and status != 'C' and status != 'O' :
                            status, response, command = getStatus( PARADIM['SESSION'], PARADIM_UI['s1']['job_id'].value, PARADIM_UI['s1'] )
                            PARADIM_UI['s1']['status'].value = "[" + k + "] " + status
                            time.sleep(5)                              
                except:
                    PARADIM_UI['s1']['job_id'].value = "Error with the PBS submittion"                    
            else :
                PARADIM_UI['s1']['status'].value =  "[" + k + "] C / cached"                
            stdin, stdout, stderr, command = PARADIM['SESSION'].execute("grep \! " + k + ".out", PARADIM_UI['s1']);
            energy = stdout[0].replace("total energy", "")
            energy = energy.replace("=", "").replace("Ry", "").replace("!", "")
            energy = float(energy.strip())
            PARADIM_UI['s2'][k].value = energy/v['units']
        PARADIM_UI['s1']['status'].value = 'C' 
    else :
        print ("ERROR: A new connection is required")


Press the "Calculate DFT Energy" button to start the tutorial, the job will be queued, run and then marked as 'C' when is complete, then follow the next step.

In [ ]:
PARADIM_UI['s1']['action'].on_click(CalculateDFTEnergy)
display (PARADIM_UI['s1']['display'])

<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><h1 style="text-align:center;padding-top:3px">Step 1: Calculate the Formation Energy</h1></div>

From the total energies of these structures we can obtain the formation energy of each reactions using the following equations,  

###### $2La_{2}O_{3} + 4NiO + O_{2} \Longleftrightarrow 4LaNiO_{3}, \qquad \Delta E = 4E(LaNiO_{3})-2E(La_{2}O_{3})-4E(NiO)- E(O_{2}) \qquad [1]$ 

###### $6La_{2}O_{3} + 8NiO + O_{2} \Longleftrightarrow 4La_{3}Ni_{2}O_{7}, \qquad \Delta E =4E(La_{3}Ni_{2}O_{7})-6E(La_{2}O_{3})-8E(NiO) -E(O_{2}) \qquad [2]$

###### $4La_{2}O_{3} + 6NiO + O_{2} \Longleftrightarrow 2La_{4}Ni_{3}O_{10}, \qquad \Delta E= 2E(La_{4}Ni_{3}O_{10})-4E(La_{2}O_{3})-6E(NiO)- E(O_{2}) \qquad [3]$

###### $La_{2}O_{3} + NiO \Longleftrightarrow  La_{2}NiO_{4}, \qquad \Delta E=2E(La_{2}NiO_{4})-2E(NiO)-2E(La_{2}O_{3}) \qquad [4]$


In [ ]:
def CalculateFormationEnergy ( event ):
    K = 0.5 #Ry->Ha
    LaNiO3 = PARADIM_UI['s2']['LaNiO3'].value
    La2O3 = PARADIM_UI['s2']['La2O3'].value
    NiO = PARADIM_UI['s2']['NiO'].value
    O2 = PARADIM_UI['s2']['O2'].value
    La3Ni2O7 = PARADIM_UI['s2']['La3Ni2O7'].value
    La4Ni3O10 = PARADIM_UI['s2']['La4Ni3O10'].value
    La2NiO4 = PARADIM_UI['s2']['La2NiO4'].value
    PARADIM_UI['s2']['DE_LaNiO3'].value = (4*(LaNiO3) - (2*(La2O3) + 4*(NiO) + (O2))) * K
    PARADIM_UI['s2']['DE_La3Ni2O7'].value = (4*(La3Ni2O7) - (6*(La2O3) + 8*(NiO) + (O2))) * K
    PARADIM_UI['s2']['DE_La4Ni3O10'].value = (2*(La4Ni3O10) - (4*(La2O3) + 6*(NiO) + (O2))) * K
    PARADIM_UI['s2']['DE_La2NiO4'].value = ((La2NiO4) - ((NiO) + (La2O3)) ) * K


Press the "Calculate Formation Energy" button to Calculate Binfing Energies

In [ ]:
PARADIM_UI['s2']['action'].on_click(CalculateFormationEnergy)
display (PARADIM_UI['s2']['display'])

<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><h1 style="text-align:center;padding-top:3px">Step 2: Thermodynamics</h1></div>

Applying $\Delta G$ for $O_2$ at equilibrium (ΔG = 0) the equilibrium temperature can be calculated : 

$\Delta G = \Delta H – T.\Delta S$  

Assume the enthalpy of formation:

$ΔH^{T=0} \approx 315 kJ/mol, S°gas,  1 bar = 205 (J⋅K^{-1}mol^{-1})$

### $LaNiO_{3} \Longrightarrow La_{2}NiO_{4}$
* $4LaNiO_{3} \qquad \Longleftrightarrow \qquad  \color{red}{2La_{2}O_{3}} + 4NiO + O_{2}  \qquad [1]$
* $\color{red}{2La_{2}O_{3}} + 2NiO  \qquad \Longleftrightarrow \qquad 2La_{2}NiO_{4}  \qquad [4]$


### $La_{4}Ni_{3}O_{10} \Longrightarrow La_{2}NiO_{4}$
* $2La_{4}Ni_{3}O_{10} \qquad \Longleftrightarrow \qquad  \color{red}{4La_{2}O_{3}} + 6NiO + O_{2}  \qquad [3]$
* $\color{red}{4La_{2}O_{3}} + 4NiO  \qquad \Longleftrightarrow \qquad 4La_{2}NiO_{4}  \qquad [4]$


### $La_{3}Ni_{2}O_{7} \Longrightarrow La_{2}NiO_{4}$
* $4La_{4}Ni_{3}O_{10} \qquad \Longleftrightarrow \qquad  \color{red}{6La_{2}O_{3}} + 8NiO + O_{2}  \qquad [2]$
* $\color{red}{6La_{2}O_{3}} + 6NiO  \qquad \Longleftrightarrow \qquad 6La_{2}NiO_{4}  \qquad [4]$


In [ ]:
def CalculateThermodynamics ( event ):
    DE_LaNiO3 = PARADIM_UI['s2']['DE_LaNiO3'].value
    DE_La3Ni2O7 = PARADIM_UI['s2']['DE_La3Ni2O7'].value
    DE_La4Ni3O10 = PARADIM_UI['s2']['DE_La4Ni3O10'].value
    DE_La2NiO4 = PARADIM_UI['s2']['DE_La2NiO4'].value

    K1 = 2#Ha->Ry
    K2 = 1312.7497 #Ry->J/mol.K
    K3 = 1000
    bar = 205
    samples = 50

    DG_LaNiO3 = (-DE_LaNiO3 + 2*DE_La2NiO4) * K1 * K2 * K3
    DG_La4Ni3O10 = (-DE_La4Ni3O10 + 4*DE_La2NiO4) * K1 * K2 * K3
    DG_La3Ni2O7 = (-DE_La3Ni2O7 + 6*DE_La2NiO4) * K1 * K2 * K3

    T_LaNiO3 = DG_LaNiO3/bar
    T_La4Ni3O10 = DG_La4Ni3O10/bar
    T_La3Ni2O7 = DG_La3Ni2O7/bar

    min_T = min(T_LaNiO3, T_La4Ni3O10, T_La3Ni2O7) - 100
    max_T = max(T_LaNiO3, T_La4Ni3O10, T_La3Ni2O7) + 100

    T = np.linspace(min_T, max_T, num=samples)

    DG_T_LaNiO3 = ((bar*T)+DG_LaNiO3) / (K3*K2*K1)
    DG_T_La4Ni3O10 = ((bar*T)+DG_La4Ni3O10) / (K3*K2*K1)#
    DG_T_La3Ni2O7 = ((bar*T)+DG_La3Ni2O7) / (K3*K2*K1)#

    data = []
    
    data.append({ 'type' : 'scatter', 'x' : T, 'y' : DG_T_LaNiO3, 'mode' : 'lines', 'name':'LaNiO3', 'legendgroup':'LaNiO3'})
    data.append({ 'type' : 'scatter', 'x' : T, 'y' : DG_T_La4Ni3O10, 'mode' : 'lines', 'name':'La4Ni3O10', 'legendgroup':'La4Ni3O10'})
    data.append({ 'type' : 'scatter', 'x' : T, 'y' : DG_T_La3Ni2O7, 'mode' : 'lines', 'name':'La3Ni2O7', 'legendgroup':'La3Ni2O7'})

    DG_P_LaNiO3 = ((bar*T_LaNiO3)+DG_LaNiO3) / (K3*K2*K1)
    DG_P_La4Ni3O10 = ((bar*T_La4Ni3O10)+DG_La4Ni3O10) / (K3*K2*K1)#
    DG_P_La3Ni2O7 = ((bar*T_La3Ni2O7)+DG_La3Ni2O7) / (K3*K2*K1)#
    
    layout = {
            'title':'Pressure / Temperature ',
            'xaxis':{'title' : 'Temperature (K)','tickvals':[round(T_LaNiO3),round(T_La4Ni3O10),round(T_La3Ni2O7)]},
            'yaxis':{'type' : 'log',
                     'tickvals':[DG_T_LaNiO3[0],DG_T_La4Ni3O10[0],DG_T_La3Ni2O7[0]], 
                     'ticktext':["LaNiO3","La4Ni3O10","La3Ni2O7"],
            },
            'showlegend':True
    }
    
    data.append({'type' : 'scatter', 'x' : [min_T,T_LaNiO3,T_LaNiO3], 'y' : [DG_P_LaNiO3,DG_P_LaNiO3,0], 'mode' : 'lines', 
             'line':{'color':"blue", 'dash' : 'dot'}, 'legendgroup':'LaNiO3', 'showlegend': False, 'name':'_LaNiO3'
    })
    data.append({'type' : 'scatter', 'x' : [min_T,T_La4Ni3O10,T_La4Ni3O10], 'y' : [DG_P_La4Ni3O10,DG_P_La4Ni3O10,0], 'mode' : 'lines', 
             'line':{'color':"orange", 'dash' : 'dot'}, 'legendgroup':'La4Ni3O10', 'showlegend': False, 'name':'_La4Ni3O10'
    })
    data.append({ 'type' : 'scatter', 'x' : [min_T,T_La3Ni2O7,T_La3Ni2O7], 'y' : [DG_P_La3Ni2O7,DG_P_La3Ni2O7,0], 'mode' : 'lines', 
             'line':{'color':"green", 'dash' : 'dot'}, 'legendgroup':'La3Ni2O7', 'showlegend': False, 'name':'_La3Ni2O7'
    })    
    fig = go.Figure(data=data, layout=layout)
    with PARADIM_UI['s3']['output']:
        clear_output()
        iplot(fig, show_link=False)



Press the "Calculate Thermodynamics" to calculate the Equilibrium Temperature, clicking on the legend will hide or show the selected material

In [ ]:
PARADIM_UI['s3']['action'].on_click(CalculateThermodynamics)
display (PARADIM_UI['s3']['display'])
display (PARADIM_UI['s3']['output'])

<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><h1 style="text-align:center;padding-top:3px">Step 3: Effect of varying oxygen partial pressure</h1></div>
Based on the Equilibrium conditions calculated on step 2 and given the following conditions:

### $R = \textbf{8.314}, \qquad P_0 = \textbf{1 atm}, \qquad \Delta G_o = (\Delta H_{T=0}$) = \textbf{-205.0 T}$

It is possible to estimate temperatures for different pressures (P)

### $\Delta G = \Delta G_o + R T ln(\frac{P}{P_0})$

### $T(P) = -\frac{\Delta G}{ 8.314 ln(P) - 205 }, \qquad T(1atm) \approx 2946K, \qquad T(300atm) \approx 3897K$

In [ ]:
def CalculateOxygenPressure ( event ):
    DE_LaNiO3 = PARADIM_UI['s2']['DE_LaNiO3'].value
    DE_La3Ni2O7 = PARADIM_UI['s2']['DE_La3Ni2O7'].value
    DE_La4Ni3O10 = PARADIM_UI['s2']['DE_La4Ni3O10'].value
    DE_La2NiO4 = PARADIM_UI['s2']['DE_La2NiO4'].value

    K1 = 2#Ha->Ry
    K2 = 1312.7497 #Ry->J/mol.K
    K3 = 1000
    bar = 205
    samples = 50
    R = 8.314
    G_0 = -205.0
    
    DG_LaNiO3 = (-DE_LaNiO3 + 2*DE_La2NiO4) * K1 * K2 * K3
    DG_La4Ni3O10 = (-DE_La4Ni3O10 + 4*DE_La2NiO4) * K1 * K2 * K3
    DG_La3Ni2O7 = (-DE_La3Ni2O7 + 6*DE_La2NiO4) * K1 * K2 * K3
      
    samples = 50
    P = np.linspace(0.1, 330, num=samples)
    P[1]=1
    
    T_LaNiO3 = -DG_LaNiO3/(R*np.log(P)+G_0)
    T_1_LaNiO3 = -DG_LaNiO3/(R*np.log(1)+G_0)
    T_300_LaNiO3 = -DG_LaNiO3/(R*np.log(300)+G_0)
    
    T_La4Ni3O10 = -DG_La4Ni3O10/(R*np.log(P)+G_0)
    T_1_La4Ni3O10 = -DG_La4Ni3O10/(R*np.log(1)+G_0)
    T_300_La4Ni3O10 = -DG_La4Ni3O10/(R*np.log(300)+G_0)
    
    T_La3Ni2O7 = -DG_La3Ni2O7/(R*np.log(P)+G_0)
    T_1_La3Ni2O7 = -DG_La3Ni2O7/(R*np.log(1)+G_0)
    T_300_La3Ni2O7 = -DG_La3Ni2O7/(R*np.log(300)+G_0)
    
    data = []
    data.append({ 'type' : 'scatter', 'x' : T_LaNiO3, 'y' : P, 'mode' : 'lines', 'name':'LaNiO3', 'legendgroup':'LaNiO3'})
    data.append({ 'type' : 'scatter', 'x' : T_La4Ni3O10, 'y' : P, 'mode' : 'lines', 'name':'La4Ni3O10', 'legendgroup':'La4Ni3O10'})
    data.append({ 'type' : 'scatter', 'x' : T_La3Ni2O7, 'y' : P, 'mode' : 'lines', 'name':'La3Ni2O7', 'legendgroup':'La3Ni2O7'})
    min_T = min(T_LaNiO3[0], T_La4Ni3O10[0], T_La3Ni2O7[0])
    
    layout = {
            'title':'Pressure / Temperature ',
            'xaxis':{'title' : 'Temperature (K)','tickvals':[ 
                round(T_1_LaNiO3),round(T_300_LaNiO3),
                round(T_1_La4Ni3O10),round(T_300_La4Ni3O10),
                round(T_1_La3Ni2O7),round(T_300_La3Ni2O7)
            ]},
            'yaxis':{'title' : 'Pressure (atm)', 'type':'log','tickvals':[1,300]},
    }
    data.append({ 'type' : 'scatter', 'x' : [min_T,T_1_LaNiO3,T_1_LaNiO3], 'y' : [1,1,0], 'mode' : 'lines', 
             'line':{'color':"blue", 'dash' : 'dot'}, 'legendgroup':'LaNiO3', 'name':'', 'showlegend': False, 
    })
    data.append({'type' : 'scatter', 'x' : [min_T,T_300_LaNiO3,T_300_LaNiO3], 'y' : [300,300,0], 'mode' : 'lines', 
             'line':{'color':"blue", 'dash' : 'dot'}, 'legendgroup':'LaNiO3', 'name':'', 'showlegend': False, 
    })
    data.append({ 'type' : 'scatter', 'x' : [min_T,T_1_La4Ni3O10,T_1_La4Ni3O10], 'y' : [1,1,0], 'mode' : 'lines', 
             'line':{'color':"orange", 'dash' : 'dot'}, 'legendgroup':'La4Ni3O10', 'name':'', 'showlegend': False, 
    })
    data.append({'type' : 'scatter', 'x' : [min_T,T_300_La4Ni3O10,T_300_La4Ni3O10], 'y' : [300,300,0], 'mode' : 'lines', 
             'line':{'color':"orange", 'dash' : 'dot'}, 'legendgroup':'La4Ni3O10', 'name':'', 'showlegend': False, 
    })
    data.append({ 'type' : 'scatter', 'x' : [min_T,T_1_La3Ni2O7,T_1_La3Ni2O7], 'y' : [1,1,0], 'mode' : 'lines', 
             'line':{'color':"green", 'dash' : 'dot'}, 'legendgroup':'La3Ni2O7', 'name':'', 'showlegend': False, 
    })
    data.append({'type' : 'scatter', 'x' : [min_T,T_300_La3Ni2O7,T_300_La3Ni2O7], 'y' : [300,300,0], 'mode' : 'lines', 
             'line':{'color':"green", 'dash' : 'dot'}, 'legendgroup':'La3Ni2O7', 'name':'', 'showlegend': False, 
    })    
    fig = go.Figure(data=data, layout=layout)
    with PARADIM_UI['s4']['output']:
        clear_output()
        iplot(fig, show_link=False)



Press the "Calculate Oxygen Pressure" button to visualize temperatures as a function of different pressures., clicking on the legend hide or show the selected material

In [ ]:
PARADIM_UI['s4']['action'].on_click(CalculateOxygenPressure)
display (PARADIM_UI['s4']['display'])
display (PARADIM_UI['s4']['output'])

<div style="height:32px;background-image:url('ParadimLogo.png');background-repeat: no-repeat;background-color:#cccccc"><h1 style="text-align:center;padding-top:3px"> Conclusion </h1></div>

If a Ruddlesden-Popper phase decomposed to 2-1-4 phase ($La_{2}NiO_{4}$) at a higher temperature value, the phase can be stabilizing at a given pressure. The higher the decomposition temperature, the lower pressure is needed to stabilize the given phase in PARADIM HPFZ furnace.
